### Manual Prompting for Compositionality with Large Language Models 

## **[ Produced by ]**

[Eunchan Lee of UNIST-LIL-LAB](https://github.com/purang2)




install
- torch
- pip install transformers

In [ ]:
''' #if your RAM down...
import gc
import torch 
  
gc.collect()
torch.cuda.empty_cache()
'''

'\nimport gc\nimport torch \n  \ngc.collect()\ntorch.cuda.empty_cache()\n'

Put datapair

In [1]:
few_src = [] 
few_tgt = []


def put_datapair(src, tgt):
  few_src.append(src)
  few_tgt.append(tgt)


# k=4
'''
put_datapair("k", "x")
put_datapair("kk", "xx")
put_datapair("kkk", "xxx")
put_datapair("kkkk", "xxxx")
'''


# k=4 -> Manual example: Text style transfer
#put_datapair(A,B): Style : A--> B
put_datapair("South Korea's Capital is Seoul", "Seoul -> South Korea")
put_datapair("Mexico's Capital is Mexico City", "Mexico City -> Mexico")
put_datapair("Germany's Capital is Berlin", "Berlin -> Germany")
put_datapair("Uganda's Capital is Kampala", "Kampala -> Uganda")



print(few_src)
print(few_tgt)


["South Korea's Capital is Seoul", "Mexico's Capital is Mexico City", "Germany's Capital is Berlin", "Uganda's Capital is Kampala"]
['Seoul -> South Korea', 'Mexico City -> Mexico', 'Berlin -> Germany', 'Kampala -> Uganda']


### Prompt

***Reference: [A Recipe for Arbitrary Text Style Transfer with Large Language Models, (Reif et al. (Google Research), ACL 2022)](https://aclanthology.org/2022.acl-short.94/)***

In [3]:
in_context_text = ''

for i in range(len(few_src)):
  #in_context_text += f'Here is some text: {few_src[i]}. Here is a rewrite of the text, which is more simple: {few_tgt[i]}\n'
  in_context_text += f'Q: {few_src[i]}. \nA: {few_tgt[i]}\n\n'

print(in_context_text)

Q: South Korea's Capital is Seoul. 
A: Seoul -> South Korea

Q: Mexico's Capital is Mexico City. 
A: Mexico City -> Mexico

Q: Germany's Capital is Berlin. 
A: Berlin -> Germany

Q: Uganda's Capital is Kampala. 
A: Kampala -> Uganda




transformers model

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline


# GPT-2 (This is the smallest version of GPT-2, with 124M parameters.) = "gpt2"
# GPT-2 (XL) (1.5B parameters.) = "gpt2-xl"
# T5 (base, with 220 million parameters.) = "t5-base"
# T5 (3B parameters.) = "t5-3b"
# T5 (11B parameters.) = "t5-11b" -> 45.2GB (Colab may not cover this size..)
# GPT-J (6B parameters.) = "EleutherAI/gpt-j-6B"
#model_zoo = ["gpt2","gpt2-xl","t5-base","t5-3b","t5-11b","EleutherAI/gpt-j-6B"]

# BLOOM ([Recommended] This is popular GPT-Like open-source model. 7B parameters.) [Decoder-only]= "bigscience/bloom-7b1"
# BLOOM (6B parameters.) [Decoder-only]= "bigscience/test-bloomd-6b3"
# T-Zero-plus-plus (popular 0-shot LM, 11B parameters.) [Encoder-decoder]= "bigscience/T0pp"
# LLAMA-7B ([Recommended] This is popular GPT-Like open-source model.but when during test i got some errors.) [Decoder-only]= "decapoda-research/llama-7b-hf"
# T-Zero (popular 0-shot LM, 11B parameters.) [Encoder-decoder]="bigscience/T0"
# In-context learning may work well at least 1 billions parameters model. -eunchan-

model_zoo = ["bigscience/bloom-7b1","bigscience/test-bloomd-6b3","bigscience/T0pp","decapoda-research/llama-7b-hf","bigscience/T0"]

#https://huggingface.co/bigscience/T0_3B


model_name = model_zoo[0]


if model_name.find('t5') > -1 or model_name.lower().find('t0') > -1: #model = t5, t0 [encoder-decoder]
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  nlg_pipeline = pipeline('text2text-generation',model=model, tokenizer=tokenizer)

else: #model = gpt,etc [decoder-only]
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(model_name)
  #nlg_pipeline = pipeline(model=model, tokenizer=tokenizer)

model inference

In [ ]:

#test_input_text = 'Q: ~~~~~~. \nA:'
test_input_text = "Italy's Capital is Roma"

test_output_length = 8 #token length  you should change for your desired output size.


if model_name.find('t5') > -1 or model_name.lower().find('t0') > -1: #model = t5, t0 [encoder-decoder]
  def generate_text(pipe, text, num_return_sequences=5, max_length=512):
    text = f"{text}"
    out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length, num_beams=5, no_repeat_ngram_size=2,)
    return [x['generated_text'] for x in out]

  #target_text = 'kjkjkjkj'
  #src_text = in_context_text + f"Here is some text: {test_input_text}. Here is a rewrite of the text, which is more simple: "
  src_text = f"Q:{test_input_text}\nA:"
  
  print("Input text:", src_text)
  test_output_text = generate_text(nlg_pipeline, src_text, num_return_sequences=1, max_length=test_output_length)


  #you can cook this output anything you want!  
  print(test_output_text)


else:  #model = GPT, T0, ETC
  #src_text = in_context_text + f"Here is some text: {test_input_text}. Here is a rewrite of the text, which is more simple: "
  src_text = f"Q:{test_input_text}\nA:"
  tokens = tokenizer.encode(src_text, return_tensors='pt')
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=len(tokens[0])+test_output_length)
  generated = tokenizer.batch_decode(gen_tokens)[0] 
  #test_output_text = generated[generated.rfind('more simple:')+12:] #here is some text: 이하를 저장
  test_output_text = generated
  print(generated)

  #you can cook this output anything you want!
  print(test_output_text)
